In [16]:
pip install -e git+https://github.com/tomquirk/realestate-com-au-api.git#egg=realestate_com_au_api


Obtaining realestate_com_au_api from git+https://github.com/tomquirk/realestate-com-au-api.git#egg=realestate_com_au_api
  Updating ./src/realestate-com-au-api clone
  Attempting uninstall: realestate-com-au-api
    Found existing installation: realestate-com-au-api 0.0.1b1
    Uninstalling realestate-com-au-api-0.0.1b1:
      Successfully uninstalled realestate-com-au-api-0.0.1b1
  Running setup.py develop for realestate-com-au-api
Note: you may need to restart the kernel to use updated packages.


In [17]:
from realestate_com_au import RealestateComAu

api = RealestateComAu()

# Get property listings
# listings = api.search(locations=["seventeen seventy, qld 4677"], channel="buy", keywords=["tenant"], exclude_keywords=["pool"])

In [14]:
listings = api.search(locations=["Barnsley, Queens Park, 6107"])


In [18]:
import pandas as pd
import numpy as np
df = pd.DataFrame(listings)

from datetime import datetime as dt
dte = dt.now().strftime('%Y%m%d')
fname = "/Users/joezhou/Downloads/TEST_REALESTATE{}.xlsx".format(dte)
df.to_excel(fname)

/Users/joezhou/opt/anaconda3/lib/python3.8/site-packages/xlsxwriter/worksheet.py:941: UserWarning: Ignoring URL 'https://www.b1homes.com.au/house-land/l16576307-st-james%3cbr/%3e%3cbr/%3eTitles%20are%20nearing,%20and%20the%20owner%20wants%20these%20lots%20sold%20ASAP.%20Time%20to%20get%20in%20to%20this%20boom%20suburb%20and%20make%20your%20offer.%20Offers%20will%20be%20considered.%20%3cbr/%3e%3cbr/%3eRanging%20from%20approximately%20207sqm%20to%20229sqm%20these%20prime%20parcels%20of%20land%20will%20lend%20themselves%20to%20those%20seeking%20to%20design%20something%20low-maintenance%20and%20ready%20to%20live%20in.%20Decide%20whether%20you%20want%20to%20secure%20street%20frontage,%20privacy%20at%20the%20rear%20or%20the%20peace%20of%20mind%20of%20a%20nice,%20central%20position%20tucked%20in%20between%20two%20other%20properties.%3cbr/%3e%20%20%3cbr/%3eThese%20exciting%20lots%20are%20nestled%20very%20close%20to%20bus%20stops,%20the%20local%20pharmacy%20and%20Albany%20Highway,%20within%20wa